In [10]:
from PIL import Image
import numpy as np 
import matplotlib.pyplot as plt
import cv2
import pandas as pd
import os

from skimage.feature import local_binary_pattern

In [17]:
data = pd.read_csv("C:\\Users\\Muhammad Ammar M\\Documents\\Kuylah S2\\Pengolahan dan Analisis Citra Digital\\PACD_Assignment\\small dataset\\MetaData.csv", index_col=False)
data.head()

,id,gender,age,county,ptb,remarks
0,1000,male,31.0,Shenxhen,0.0,normal
1,1001,male,64.0,Shenxhen,0.0,normal
2,1002,male,35.0,Shenxhen,0.0,normal
3,"1003,male,32,Shenxhen,1,""STB,ATB,tuberculosis ...",NaN,NaN,NaN,NaN,NaN
4,1004,male,2.0,Shenxhen,1.0,secondary PTB in the bilateral upper field


In [7]:
class utils():
    def __init__(self):
        pass
    
    @staticmethod
    def display_image(title, image):
        cv2.imshow(title, image)
        cv2.waitKey(0)
        cv2.destroyAllWindows()

    @staticmethod
    def display_histogram(image):
        plt.hist(np.array(image).flatten(), bins=256, color='gray')
        plt.title('Histogram')
        plt.xlabel('Intensitas')
        plt.ylabel('Frekuensi')
        plt.show()

    @staticmethod
    def convert_data_to_gray(path):
        folder_path = os.path.join(path, 'image')
        
        if os.path.isdir(folder_path):
            for file in os.listdir(folder_path):
                img_path = os.path.join(folder_path, file)
                
                if os.path.isfile(img_path):
                    img = cv2.imread(img_path)
                    if img is None:
                        print(f"{file} : None")
                    else:
                        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
                        cv2.imwrite(img_path, gray)
                        # print(f"{file} : Non None")

    @staticmethod
    def dataset_check(path):
        folder_path = os.path.join(path, 'image')

        if os.path.isdir(folder_path):
            for file in os.listdir(folder_path):
                img_path = os.path.join(folder_path, file)

                img = Image.open(img_path)
                img = np.array(img)
                
                if img is None:
                    print(f"{file}: Cannot read file")
                    continue

                # Cek jumlah channel
                if len(img.shape) == 2:
                    # print(f"{img_path} pass")
                    continue
                elif len(img.shape) == 3 and img.shape[2] == 3:
                    print(img.shape)
                    print(f"{img_path} has 3 dimenstion")
                    return 0
                else:
                    return 0    
        else:
            print("Folder Kosong")
        
        return 1

class preprocess():
    def __init__(self):
        self.preprocessed_folder = 'preprocessed_img'
        self.image_folder = 'image'
        self.mask_folder = 'mask'
    
    def apply_mask(self, image, mask):
        mask_bin = (mask > 0).astype(np.uint8)
        masked_image = image * mask_bin
        return masked_image
    
    def crop(self, image, mask):
        ys, xs = np.where(mask > 0)

        if len(xs) == 0 or len(ys) == 0:
            raise ValueError("Mask tidak valid")
        
        xmin, xmax = np.min(xs), np.max(xs)
        ymin, ymax = np.min(ys), np.max(ys)

        cropped_image = image[ymin:ymax, xmin:xmax]
        return cropped_image
    
    def resize(self, image, target=(512, 512)):
        h, w = image.shape[:2]
        target_w, target_h = target

        scale = min(target_w / w, target_h / h)
        new_w = int(w * scale)
        new_h = int(h * scale)

        resized = cv2.resize(image, (new_w, new_h))

        pad_x = target_w - new_w
        pad_y = target_h - new_h

        padded = cv2.copyMakeBorder(
            resized,
            pad_y // 2, pad_y - pad_y // 2,
            pad_x // 2, pad_x - pad_x // 2,
            cv2.BORDER_CONSTANT,
            value=0
        )
        return padded

    def clahe(self, image):
        clahe = cv2.createCLAHE(clipLimit=5)
        clahe_img = np.clip(clahe.apply(image) + 30, 0, 255).astype(np.uint8)
        # _, threshold_img = cv2.threshold(image, 155, 255, cv2.THRESH_BINARY)
        return clahe_img
    
    def median_blur(self, image):
        blured_image = cv2.medianBlur(image, 5)
        return blured_image
    
    def preprocessing(self, folder_path):
        for file in os.listdir(folder_path):
            print(file)

        processed_image_folder = os.path.join(folder_path, self.preprocessed_folder)

        if not os.path.exists(processed_image_folder):
            os.makedirs(processed_image_folder)
            print(f"Created {processed_image_folder}")

        image_folder = os.path.join(folder_path, self.image_folder)
        mask_folder = os.path.join(folder_path, self.mask_folder)

        if not os.path.isdir(image_folder):
            print("Folder image tidak ditemukan!")
            return

        for file in os.listdir(image_folder):
            img_path = os.path.join(image_folder, file)
            mask_path = os.path.join(mask_folder, file)

            if not os.path.isfile(img_path):
                continue

            img = Image.open(img_path)
            mask = Image.open(mask_path)
            img = np.array(img)
            mask = np.array(mask)

            masked_image = self.apply_mask(img, mask)
            cropped_image = self.crop(masked_image, mask)
            resized_image = self.resize(cropped_image, (1024, 1024))
            clahe_result = self.clahe(resized_image)
            median_blur_result = self.median_blur(clahe_result)

            pre_normalized = median_blur_result.astype(np.float32)
            normalized = pre_normalized / 255.0

            save_img = (normalized * 255).astype(np.uint8)
            output_path = os.path.join(processed_image_folder, file)
            cv2.imwrite(output_path, save_img)
            # np.save(output_path.replace('.png', '.npy'), normalized)
            # print(f"Saved: {output_path}")

class featureExtract():
    def __init__(self):
        self.preprocessed_folder = 'preprocessed_img'
        # lbp
        self.radius = 2
        self.n_points = 8 * self.radius

    def lbp(self, image):
        lbp = local_binary_pattern(image, self.n_points, self.radius, method='uniform')
        hist, _ = np.histogram(lbp.ravel(), bins=np.arange(0, self.n_points + 3), range=(0, self.n_points + 2))
        hist = hist.astype(float)
        hist /= hist.sum()
        return hist
    
    def feature_extract(self, folder_path):
        image_folder = os.path.join(folder_path, self.preprocessed_folder)
        if os.path.isdir(image_folder):
            for file in os.listdir(image_folder):
                img_path = os.path.join(image_folder, file)
                img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)

                lbp_result = self.lbp(img)
                print(len(lbp_result))




In [3]:
path = 'C:\\Users\\Muhammad Ammar M\\Documents\\Kuylah S2\\Pengolahan dan Analisis Citra Digital\\PACD_Assignment\\small dataset\\ChestXRay\\ChestXRay\\'
# utils.convert_data_to_gray(path)
ok = utils.dataset_check(path)
if ok:
    print("Dataset ready to use")
else:
    print("Dataset unready")

Dataset ready to use


In [4]:
prep = preprocess()
prep.preprocessing(path)

image
mask
Created C:\Users\Muhammad Ammar M\Documents\Kuylah S2\Pengolahan dan Analisis Citra Digital\PACD_Assignment\small dataset\ChestXRay\ChestXRay\preprocessed_img


In [8]:
fe = featureExtract()
fe.feature_extract(path)

18
18
18
18
18
18
18
18
18
18
18
18
18
18
18


In [6]:
# Load image
img = Image.open("C:\\Users\\Muhammad Ammar M\\Documents\\Kuylah S2\\Pengolahan dan Analisis Citra Digital\\PACD_Assignment\\small dataset\\ChestXRay\\ChestXRay\\preprocessed_img\\1000.png")
img = np.array(img)

# Inspeksi Gambar
# utils.display_image("Test", img)
# utils.display_histogram(img)
# print(img.shape)